In [2]:
import numpy as np
from collections import namedtuple
from ortools.sat.python import cp_model
import timeit
import random

In [3]:
''' Algorithmic Approach : use solve_schedule_algo(intervals, num_resources) '''

# Proof: http://www.cs.toronto.edu/~milad/csc373/lectures/T1.pdf
# O(n^2)
def solve_schedule_algo(intervals, num_resources):
  sorted_end   = sorted(intervals, key=lambda x: x[1])    # n*log(n)
  schedules = [[] for _ in range(num_resources)]

  for interval in sorted_end: 
    # get schedule with smallest end time such that the start time of interval is greater than endtime
    index = assign_schedule(interval[0], schedules)
    if index > -1: schedules[index].append(interval)

  return sum([len(schedule) for schedule in schedules]), schedules  

def assign_schedule(start_time, schedules):
  index = -1
  latest_end = -1
  for i, schedule in enumerate(schedules): 
    schedule_end_time = 0 if len(schedule) < 1 else schedule[-1][1]
    if schedule_end_time > start_time:
      continue
    elif schedule_end_time > latest_end:
      latest_end = schedule_end_time
      index = i
  return index

In [4]:
''' SMT Approach : use solve_schedule_smt(intervals, num_resources) '''

def solve_schedule_smt(intervals, num_resources):
  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Maximize Length of Schedule
  model.Maximize(sum(assign_resource.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return solver.ObjectiveValue(), solution

In [5]:
''' SMT Approach + Resource Optimization : use solve_schedule_opt(intervals, num_resources) '''

def solve_schedule_opt(intervals, num_resources):
  # Solve Intermediate Solution
  obj = int(solve_schedule_smt(intervals, num_resources)[0])

  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  durations = np.zeros((num_resources, num_intervals))
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      durations[r][i] = interval[1]-interval[0]
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Add Optimal Value for Solver 2
  model.Add(sum(assign_resource.flatten()) == obj)
  model.Maximize(cp_model.LinearExpr.WeightedSum(assign_resource.flatten(), durations.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return obj, solution, solver.ObjectiveValue()

In [6]:
''' Multiple-Objective Optimization Static Weighted : solve_static_weighted(intervals, num_resources, weight) '''

def solve_static_weighted(intervals, num_resources, weight=10000):
  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  durations = np.zeros((num_resources, num_intervals))
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      durations[r][i] = interval[1]-interval[0]
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Weighted Sum for # of tasks + resource utilization time
  model.Maximize(weight*sum(assign_resource.flatten())+cp_model.LinearExpr.WeightedSum(assign_resource.flatten(), durations.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  num_scheduled = 0
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        num_scheduled += is_scheduled
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return num_scheduled, solution, solver.ObjectiveValue()

In [7]:
''' Multiple-Objective Optimization Dynamic Weighted : solve_dynamic_weighted(intervals, num_resources) '''

def solve_dynamic_weighted(intervals, num_resources):
  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # Grab Maximum Range of Intervals
  flattened = list(sum(intervals,()))
  task_weights = max(flattened) - min(flattened)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  durations = np.zeros((num_resources, num_intervals))
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      durations[r][i] = interval[1]-interval[0]
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Weighted Sum for # of tasks + resource utilization time
  model.Maximize(task_weights*sum(assign_resource.flatten())+cp_model.LinearExpr.WeightedSum(assign_resource.flatten(), durations.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  num_scheduled = 0
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        num_scheduled += is_scheduled
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return num_scheduled, solution, solver.ObjectiveValue()

In [8]:
def generate_intervals(num_intervals, start=0, stop=1440):
  intervals = []
  for i in range(num_intervals):
    x = random.randint(start, stop)
    y = random.randint(start, stop)
    if x == y:
      i -= 1
    elif x < y: 
      intervals += [(x, y)]
    else: 
      intervals += [(y,x)]
  return intervals


In [9]:
def check_testcase(testcase, approach1, approach2, label1="Approach 1", label2="Approach 2", output=True):
  output1 = approach1(*testcase)
  output2 = approach2(*testcase)
  sol1, sch1 = output1[0], output1[1]
  sol2, sch2 = output2[0], output2[1]

  # Ensure that optimal value matches 
  assert(sol1 == sol2)

  if output:
    print(f'Intervals: {str(testcase.intervals)}\nNumber of Resources: {testcase.num_resources}\n')
    print(f'{label1}\tTotal Time: {cumulative_time(sch1)}')
    display_results(sch1)
    print('--------')
    print(f'{label2}\tTotal Time: {cumulative_time(sch2)}')
    display_results(sch2)

  return output1, output2

testcase = namedtuple('testcase', ['intervals', 'num_resources'])

In [10]:
def cumulative_time(schedules): 
  total_time = 0
  for intervals in schedules: 
    for i in intervals: 
      total_time += i[1]-i[0]
  return total_time

def display_results(schedules): 
  for i, s in enumerate(schedules):
    print(f'Resource {i+1}:', s)

def get_runtime(approach, intervals, num_resources):
  start = timeit.default_timer()
  res = approach(intervals, num_resources)
  end = timeit.default_timer()
  return end-start, res

## Correctness
Tests to show correctness of SMT solver. If incorrect, assertion should fail. 

In [ ]:
# Comparing Algorithmic vs. SMT Output (Simple Cases)

testcases = [
  # testcase([(1,3), (4,11), (2,5), (6,8), (9,12), (7,10)], 1),
  testcase([(1,3), (4,11), (2,5), (6,8), (9,12), (7,10)], 2), 
  # testcase([(1,3), (4,11), (2,5), (6,8), (9,12), (7,10)], 3),
  # testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 1),
  # testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 2),
  testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 3),
  # testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 4),
  testcase([(1,10), (1,15), (11,20), (16,20), (15,21)], 2),
]
for case in testcases: 
  check_testcase(case, solve_schedule_algo, solve_schedule_smt, "Algorithmic", "SMT Solver")
  print('--------')
  # print('Algorithmic Runtime: ', get_runtime(solve_schedule_algo, *case))
  # print('SMT Synthesis Runtime: ', get_runtime(solve_schedule_smt, *case))
  print('\n.....\n')


In [ ]:
# Comparing Algorithmic vs. SMT Output (Randomly Generated)
num_tests = 10

for i in range(num_tests):
  num_intervals = random.randint(10,25)
  num_resources = random.randint(1, 5)
  intervals = generate_intervals(num_intervals)
  case = testcase(intervals, num_resources)
  try:
    check_testcase(case, solve_schedule_algo, solve_schedule_smt, "Algorithmic", "SMT Solver", False)
    print(f"Case {i+1:2}: PASSED")
  except: 
    print(f"Case {i+1:2}: FAILED")
    print(case)
  

In [ ]:
# Comparing Algorithmic vs. Dual SMT (Randomly Generated)
num_tests = 10

for i in range(num_tests):
  num_intervals = random.randint(10,25)
  num_resources = random.randint(1, 5)
  intervals = generate_intervals(num_intervals)
  case = testcase(intervals, num_resources)
  try:
    check_testcase(case, solve_schedule_algo, solve_schedule_opt, "Algorithmic", "SMT Solver", False)
    print(f"Case {i+1:2}: PASSED")
  except: 
    print(f"Case {i+1:2}: FAILED")
    print(case)
  

In [ ]:
# Resource Utilization Optimization
testcases = [
  testcase([(1,3), (4,11), (2,5), (6,8), (6,9), (9,12), (7,10), (7,12)], 2), 
  testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 1),
  testcase([(1,10), (1,15), (11,20), (16,20), (15,18), (15,21)], 2),
]

for case in testcases:
  sol_alg, sch_alg = solve_schedule_algo(*case)
  sol_dual, sch_dual, obj_dual = solve_schedule_opt(*case)
  sol_wgt, sch_wgt, obj_wgt = solve_static_weighted(*case)

  assert(sol_alg == sol_dual)
  assert(sol_dual == sol_wgt)

  time_alg = cumulative_time(sch_alg)
  time_dual = cumulative_time(sch_dual)
  time_wgt = cumulative_time(sch_wgt)

  assert(time_alg <= time_dual)
  assert(time_dual == time_wgt)

  print(f"Intervals: {case.intervals}")
  print(f"Number of Resources: {case.num_resources}\n")

  print(f'Algorithm\tTotal Time: {time_alg}')
  display_results(sch_alg)
  print("--------")
  print(f'Dual SMT\tTotal Time: {time_dual}')
  display_results(sch_dual)
  print("--------")
  print(f'Weighted SMT\tTotal Time: {time_wgt}')
  display_results(sch_wgt)

In [40]:
# Static v. Dynamic

intervals = [(1,3), (3,5), (5,7), (1, 1000000000000), (2, 4), (4, 6), (6, 11), (2, 1000000000000)]
num_resources = 2

static_res = solve_static_weighted(intervals, num_resources)
dynamic_res = solve_dynamic_weighted(intervals, num_resources)

print(f'Static Weighted SMT\tTotal Time: {cumulative_time(static_res[1])}')
display_results(static_res[1])
print("--------")
print(f'Dynamic Weighted SMT\tTotal Time: {cumulative_time(dynamic_res[1])}')
display_results(dynamic_res[1])


Static Weighted SMT	Total Time: 1999999999997
Resource 1: [(1, 1000000000000)]
Resource 2: [(2, 1000000000000)]
--------
Dynamic Weighted SMT	Total Time: 15
Resource 1: [(1, 3), (3, 5), (5, 7)]
Resource 2: [(2, 4), (4, 6), (6, 11)]


## Timed Trials

Code to run timed trials for performance evaluation.

In [ ]:
# Change Number of Intervals - Single Method

method = solve_schedule_algo
min_intervals = 1
max_intervals = 500
num_resources = 1

for i in range(min_intervals, max_intervals+1):
  intervals = generate_intervals(i)
  print(get_runtime(method, intervals, num_resources)[0])

In [ ]:
# Change Number of Resources - Single Method

method = solve_schedule_smt
max_resources = 15
num_intervals = 30

# Use to Estimate Max Resources
# alg_sol, alg_sch = solve_schedule_algo(intervals, num_resources)
# print(alg_sol)
# display_results(alg_sch)

intervals = generate_intervals(num_intervals)
for i in range(1, max_resources+1):
  print(get_runtime(method, intervals, i)[0])

In [ ]:
# Bottleneck Approximation
num_resources = 19
size = 15           # starting test size

runtime = 0
while runtime < 30:
  size += 1
  intervals = generate_intervals(size)
  runtime = get_runtime(solve_schedule_smt, intervals, num_resources)[0]
  print(size, runtime, sep="\t")


## Variations

Proof of concept implementation & basic example of GISMP variation of ISMP problem. 

In [29]:
''' GISMP (Group Interval Scheduling Maximization Problem) '''

# group_ranges : array of the index of the last interval in group (non-inclusive)
# ex. group_ranges = [3, 6, 9, 10]
# Group 1 : intervals[:3], Group 2 : intervals[3:6], etc. 
def solve_gismp(intervals, num_resources, group_ranges):
  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each group scheduled at most once
  i = 0
  for g in group_ranges: 
    model.AddAtMostOne(assign_resource[:,i:g].flatten())
    i = g

  # Maximize Length of Schedule
  model.Maximize(sum(assign_resource.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return solver.ObjectiveValue(), solution

In [33]:
# Example w/ 1 Group : only 1 is scheduled
intervals = [(1,3), (4,11), (2,5), (6,8), (6,9), (9,12), (7,10), (7,12)]
num_resources = 1
groups = [8]
# groups = range(1,9)

solve_gismp(intervals, num_resources, groups)

(1.0, [[(1, 3)]])